# Astro 528: Lab 2, Exercise 3

## Model for Radial Velocity of a Star with no Planet

In this exercise, we will consider a radial velocity planet search that measures the velocity of a target star with independent, uncorrelated, Gausa measurement uncertainties.  First, I'll provide some code for the simplest possible model, one where the star has no planets and its true velocity is a constant.  I'll do it in a way that lends itself to generalizing.  Then you'll write code for a model with one planet on a circular orbit.  Then, you'll compute the likelihood of the observations a few different ways.

In [ ]:
"Parameters for a radial velocity model consisting of a constant radial velocity"
struct rv_model_const
   mean_rv::Float64
end

"Compute the radial velocity at specified time using rv_model_const model."
function (model::rv_model_const)(time)  
   return model.mean_rv
end

"""
   generate_simualted_data(times, sigma, model)

Generate simulated observations assuming Gaussian measurement noise.

Inputs:
- times: Array of observation times
- sigma: Array specifying standard deviation of observations
- model: User-provided model.  Must provide a function that takes a scalar time
         as input and return predicted velocity.

Output:  Array of simulated observations
"""
function generate_simualted_data(model, times::Array, sigmas::Array)
    @assert length(times) == length(sigmas)
    @assert length(times) >= 1
    @assert all(isfinite.(times))
    @assert all(isfinite.(sigmas))
    model.(times) .+ sigmas .* randn(length(sigmas))
end

Note that `generate_simulated_data()` uses [`randn()`](https://docs.julialang.org/en/v1/stdlib/Random/#Base.randn) to draw random variables from the standard normal distribution.  Next, I'll demonstrate calling the functions above.

In [ ]:
rv_model = rv_model_const(3.0) # Create a model to return a constant RV of 3m/s.
time_jan1_2019 = 2458484.5     # Julian date for 0UT on Jan 1, 2019
rv_model(time_jan1_2019)       # Evaluate model at specified time

Let's assume a measurement precision of $\sigma_i = 2$ m/s at each of $N_{obs}$ well-separated observation times.

In [ ]:
n_obs = 5
obs_data = generate_simualted_data(rv_model, ones(n_obs), 2.0*ones(n_obs) )

Now, we'll import the test module (that is part of the standard Julia library) and preform a few simple tests.

In [ ]:
# Import the test module that is part of the standard Julia library and 
using Test
@test rv_model_const(42)(1.0) == 42.0  
@test rv_model_const(3.14159)(17.0) == 3.14159

Testing equality or inequality is straightforward for integers, but dangerous for floating point numbers.  If a floating point number is equal to an integer, then it's generally ok, but I think it's better to always be cautious about testing floating  point numbers for equality.  Instead, you can test that two numbers are approximately equal using $\simeq$.  In Jupyter or Atom (and probably many other IDEs), you can get unicode characters like $\simeq$ by typing `\approx<tab>`.   When testing that two numbers are approximately equal you need to specify a tolerance.  `atol` refers to an absolute tolerance, while `rtol` refers to a relative or fractional tolerance.  
For further information on using these, see the [Julia manual](https://docs.julialang.org/en/v1/stdlib/Test/index.html#Basic-Unit-Tests-1).


Explore how Julia's Test.@test macro behaves by running and tinkering with the following code.

In [ ]:
@test 10 == 10
@test 10 != 11
@test 10.0 ≈ 10.001 atol=0.01     
@test 10.0 ≈ 10.001 rtol=0.0001
@test 10.0 ≈ 10.001 rtol=0.0001
#@test 10.0 ≈ 10.001 atol=0.0001  # This test would fail
#@test 10.0 ≈ 10.001   # This test would fail due to default tolerance being smaller

## Model for Radial Velocity of a Star with a Planet on a Circular Orbit 

### Create a type to contain model parameters
A model to describe the radial velocity of a star with planet will require more model parameters.  We could simply use an array of floating point values.  But that can be risky, since it would be easy to confuse the order of parameters.  One way to help reduce the chance of such a bug is to define your own custom type.  In addition to storing data, making it a type will make it possible for the compiler to recognize errors (if you try to pass a variable of the wrong type to a function) and to optimize functions.  

(Another advantage of a custom type is that it can aid reusability of code, even if low-level details are refactored long after the original code was developed, if the custom type is accessed via a set of documented functions, rather than by accessing its contents directly.  We'll come back to this later.)  

a.  Write a [composite type](https://docs.julialang.org/en/v1/manual/types/index.html#Composite-Types-1) named `rv_model_circ` that consists of the orbital period, radial velocity amplitude of the star, the time of the star's maximum radial velocity and the star's time-averaged radial velocity.  (FYI, for historical reasons, observational astronomers define positive velocity as moving away from the observer.)

In [ ]:
# INSERT CODE

When a user creates a compound type, Julia automatically creates a function that can be used to intialize a variable of that type.  In this case it would be `rv_model_circ(Float,Float,Float,Float)`.  The order of the function arguments matches the order of the types when the user defined the type.  Often, most of the variables have different types, and this can work well.  However, when there are several variables with the same type, this can be a little dangerous.  It would be easy for someone to confuse the order of the parameters (or even the choice of parameterization).  One strategy for reducing this risk is to create a constructor function that takes named parameters.  For example, for our `rv_model_const` type, we could make the following constructor function.

In [ ]:
function rv_model_const(;mean_rv=NaN,unused_parameter=1.0)
    @assert !isnan(mean_rv)
    rv_model_const(mean_rv)
end
rv_model_const(mean_rv=3)

Notice that a named parameter needs a default value.  In some cases (for parameters that truely are optional), it makes sense to pick a good default value.  In this case, we don't really want the mean_rv to be optional.  So I've set the default value of `mean_rv` to NaN and include an assertion to make sure that `mean_rv` has some other value.  

b.  Write a constructor function for your `rv_model_circ` type using named parameters to reduce risk of someone accidentally initializing it incorrectly.  For names, use `P` for period, `K` for the amplitude, `t_rv_max` for the time of maximum radial velocity, and `mean_rv` for the time_averaged velocity.

In [ ]:
# INSERT CODE

As we've just seen, Julia allows there to be multiple functions with the same name.  The choice of which function to call is determined by the type of the variables passed to the function.  Julia features "multiple dispatch", meaning that the choice of what function to call depends on all the function arguments, rather than just the first one.  Multiple dispatch can be powerful for allowing programmers to write generic and highly optimized code.  

To harness that full power it is useful to use a functional programming paradigm.  However, Julia can also be used to express object-oriented programming patterns, that may be more familiar/comfortable to programmers used to traditional object-oriented languages (e.g., C++, Java, Python).  As one example, if you like the pattern of using an "object", then you can "overload" the parenthesis operator for a user-defined composite type.  The syntax for "overloading" the () operator isn't obvious, so I've provided the shell below.

c. Write a function that allows a variable of type `rv_model_circ` to be called as if it were a function taking a single input, the time at which to evaluate the model, and returning a single value, the radial velocity predicted given the model parameters contained in the variable of type `rv_model_circ`.

In [ ]:
# INSERT CODE

### Add Assertions

Sometimes a programmer calls a function with arguments that either don't make sense or represent a case that the function was not originally designed to handle properly. The worst possible function behavior in such a case is returning an incorrect result without any warning that something bad has happened. Returning an error at the end is better, but can make it difficult to figure out the problem. Generally, the earlier the problem is spotted, the easier it will be to fix the problem. Therefore, good developers often include assertions to verify that the function arguments are acceptable.  

For example, in `generate_simualted_data` above, we included assertions that the size of the arrays for observations times measurement uncertainties matched.  We also checked that there was at least one observation.  

d.  What are the preconditions for your your function `rv_model_circ(time)`?  
Use these to motivate assertions for your function overloading the `()` operator for `rv_model_circ`.  Update your code to include at least one assertion for each input parameter.

### Write unit tests for your function
e.  Use the preconditions above to write at least three unit tests for `rv_model_circ(time)`.

In [ ]:
# INSERT CODE

Run your tests.  Do your functions pass all of them?  If not, correct the function (and tests if necessary) and rerun the tests.  


### Automate the application of your tests.
Often, a well-intentioned programmer introduces a bug, but doesn't notice until long after the bug was written.  One way to reduce the risk of such bugs is to have an comprehensive set of unit tests that are applied _automatically_ each time a developer commits a change.  If some new codes causes a test to fail, we want to know that promptly, so it can be fixed and before it causes scientists to lose time running the buggy code or trying to interpret results of a buggy code.

The '.travis.yml' file provided in this repository already provides instructions for [Travis-CI.com](https://travis-ci.com/) to automatically run tests each time you commit changes and push them to GitHub.com.  (You may need to log into [Travis-CI.com](https://travis-ci.com/) and give it permission to access your repository first.)  The tests for this notebook are in `tests/test3.jl`.  

f.  Add your tests above to `tests/test3.jl`, so that they become part of your repository's _continuous integration_ testing. 

### Testing the assertions!
In this case, the assertions are probably pretty simple.  But sometimes, the assertions can be complicated enough that you'll need to test that they're working as intended!  When an assert statement is followed by an expression that evaluates to false, then it ["throws an exception"](https://docs.julialang.org/en/v1.0/manual/control-flow/#Exception-Handling-1).  We'll want to make sure that our code is throwing an exception when we pass our function invalid arguments.  For example, let's test that `generate_simulated_data` throws an exception in the following cases.

In [ ]:
@test_throws AssertionError  generate_simualted_data(rv_model, ones(10), ones(11))
@test_throws AssertionError  generate_simualted_data(rv_model, fill(Inf,10), ones(10))
@test_throws AssertionError  generate_simualted_data(rv_model, ones(10), fill(NaN,10))

g.  Write a test that makes sure your assert statement for `rv_model_circ` is indeed throwing an assertion when the inputs are not valid.  

# Validating a model

While unit tests are great for identifying issues with small pieces of code, sometimes bugs arise because of how units are combined.  Or maybe scientifically interesting issue only becomes apparent once you couple different parts of your code.  
Another important strategy is to validate that your code performs as expected on simulated data.  Of course, actual data is probably more complicated than your simulated data.  But if your code doesn't work on simulated data, then it's very unlikely to work well when applied to realistic data.

## Generate simulated dataset
h.  Generate an ordered list of 100 simulated observation times drawn uniformly during calender year 2019 (expressed as Julian date) and store in a variable named `obs_times`.  You'll likely want to use the functions [`rand(min:max,number)`](https://docs.julialang.org/en/v1/stdlib/Random/#Base.rand) and [`sort`](https://docs.julialang.org/en/v1/base/sort/#Base.sort) or [`sort!`](https://docs.julialang.org/en/v1/base/sort/#Base.sort!).

In [ ]:
# INSERT CODE

i.  Now, combine the `generate_simualted_data` function above with your `rv_model_circ` type and the function above to generate simulated data corresponding to an orbital period of 4 days, amplitude of 10 m/s, the maximum velocity (away from the observer) occurring on Jan 1, 2019, and a time-averaged velocity of 3 m/s.  Include uncorrelated Gaussian measurement noise with standard deviation 2 m/s.
the circular model at 100 times with Julian dates during the 2019 calender year.  Store the result in `obs_rvs`.

In [ ]:
# INSERT CODE

At this point, it is often useful to visually inspect your simulated data to make sure that your functions are doing what you expect.

In [ ]:
using Plots   # In separate cell, since only need to run once per notebook

In [ ]:
plt_raw = scatter(obs_times,obs_rvs,yerror=2*ones(length(obs_times)), 
    xlabel ="Time (JD)", ylabel="RV (m/s)", legend=:no)
plt_phased = scatter(mod.(obs_times.-time_jan1_2019,4.0),obs_rvs,yerror=2*ones(length(obs_times)), 
    xlabel ="Time (d)", ylabel="RV (m/s)", legend=:no)
plot(plt_raw,plt_phased,layout=(2,1))

j.  Are you happy with the behavior of your functions?  
If you found a bug, then create a new unit test that would have identified that bug and add it to the 'tests/test2.jl' file.

# Computing Likelihood of Data

A common task in astronomy is to compute the probability of obtaining a set of observations given an assumed model.  For example, consider a set of observations ($y_i$'s), each of which can be assumed to follow a normal distribution centered on the true value ($z_i$) with a standard deviation of $\sigma_i$, so 

$$p(y_i | z_i) = \frac{\exp \left[-(y_i-z_i)^2/(2\sigma_i^2)\right]}{\sqrt{2\pi \sigma_i^2}}$$

When the measurement error for each observation is independent and uncorrelated with the other observations, the probability of a combination of measurements is simply the product of the individual probabilities.  

### Write a function for likelihood of one observation
k.  Write a function `gauss_std_normal` to calculate the probability density of a random variable drawn from a standard normal distribution (i.e., Gaussian with mean zero and standard deviation of unity).  
(FYI: You can use the built-in functions `exp(x)` and `sqrt(x)`.  Julia makes it easy to define small functions using the syntax: `add3(a,b,c) = a+b+c` .  )

In [ ]:
# INSERT CODE

### Write a function for likelihood of a set of observations
l.  Write a function `likelihood` that takes inputs of an `rv_model_circ`, followed by arrays of observation times, observed RVs, measurement uncertainties.  It should  return the likelihood of the observations, assuming that the measurement are independent an uncorrelated.  Therefore, the likelihood of the observed data is simply the product of the probability of each observation.

In [ ]:
# INSERT CODE

likelihood(model,obs_times,obs_rvs,2*ones(length(obs_times)))

### Combine the above into a function
m.  Write a function `simulate_rvs_and_calc_likelihood` that takes a single integer parameter ($N_{obs}$) and returns the likelihood of a set of simulated observations using the model parameters specified above.  To facilitate testing, seed the random number generator at the beginning of each function call using  [`Random.seed!()`](https://docs.julialang.org/en/v1/stdlib/Random/index.html#Generators-(creation-and-seeding)-1).

In [ ]:
using Random
function simulate_rvs_and_calc_likelihood(num_obs::Integer)
    Random.seed!(42)
    # INSERT CODE
end

### Test your function
n.  While the specific results depend on the values of the pseudo-random numbers generated, we should still think about what results we expect.  What is the expected value of the log likelihood as a function of $N_{obs}$?  What is a reasonable expectation for how much the results would vary depending on the specific pseudo-random numbers used?
(Tip:  It may help to consider the properties of the [$\chi^2$ distribution](https://en.wikipedia.org/wiki/Chi-squared_distribution).)

o.  Test your function `simulate_rvs_and_calc_likelihood` for $N_{obs}$ = 100 and $N_{obs}$ = 600.  Are you happy with the results?  If not, what is going wrong?

In [ ]:
println("N_obs = 100: L = ",simulate_rvs_and_calc_likelihood(100))
println("N_obs = 600: L = ",simulate_rvs_and_calc_likelihood(600))

INSERT RESPONSE

### Write functions for log likelihood
p. Write functions `log_gauss_std_normal`, `log_likelihood` and `simulate_rvs_and_calc_log_likelihood` analogous to those above, except now calculating the log likelihood (base e).

In [ ]:
# INSERT CODE

### Test your functions
q.  Test your function `simulate_rvs_and_calc_log_likelihood` for $N_{obs}$ = 100 and $N_{obs}$ = 600.  Compare the results of this function with the results of your `simulate_rvs_and_calc_likelihood` function and your theoretical expectations for the log likelihood of each dataset.

In [ ]:
#INSERT CODE
println("N_obs = 100: log L = ",simulate_rvs_and_calc_log_likelihood(100))
println("N_obs = 600: log L = ",simulate_rvs_and_calc_log_likelihood(600))

INSERT REPSONSE

### Conclusion
q.  What lessons does this exercise illustrate that could be important when writing code for your research?  

INSERT RESPONSE